# Set Up for Project Imports


In [25]:
import sys
from pathlib import Path

In [26]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [27]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

# Imports

In [78]:
from data.api import UcIrvineAPI, BureauEconomicAnalysisAPI
from data.dataset_id import DatasetIDs
import json
import pandas

In [29]:
pandas.set_option('display.max_colwidth', None) # show all text in cells

# UcIrvine Data

In [79]:
uci_df = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

/Users/isaacpacheco/Documents/School/Graduate/Fall 2025/CS 577 - Data Science/Project/Github/.venv/lib/python3.13/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


# Bureau of Economic Data

In [180]:
bea_df = BureauEconomicAnalysisAPI.fetch_dataset('Regional', GeoFips = 'STATE', TableName = 'SARPP', Year = '2019', LineCode = '1')